In [1]:
import pandas as pd
import time
import requests
import json
from moralis import evm_api
import re

In [5]:
votes = pd.read_csv('unicef_grant_votes.csv')
votes.rename(columns={'amount': 'funding_amount','created_at':'Funded_at'}, inplace=True)
votes['No_Grants_Funded'] = votes.groupby('source_wallet')['source_wallet'].transform('count')


In [ ]:

def get_transactions_and_date_info(address):
    dates = []
    api_key = "api-key"
    params = {
        "address": address, 
        "chain": 'eth',
        }

    txn_history = evm_api.transaction.get_wallet_transactions(
        api_key=api_key,
        params=params,
    )
    txn_history = txn_history['result']
    for index in range(len(txn_history)):
        date = txn_history[index]['block_timestamp']
        trimmed = re.search(r'\d{4}-\d{2}-\d{2}', date).group(0)
        dates.append(trimmed)

    dates = [*set(dates)]

    first_active_date =  re.search(r'\d{4}-\d{2}-\d{2}',txn_history[-1]['block_timestamp']).group(0)

    last_active_date = re.search(r'\d{4}-\d{2}-\d{2}',txn_history[0]['block_timestamp']).group(0)

    inital_ammount_recieved = int(txn_history[-1]['value'])/10**18

    last_trasaction_out = txn_history[0]['to_address']
    
    return len(dates),inital_ammount_recieved,last_trasaction_out,first_active_date,last_active_date

def get_txn_count(address):
    pokt_eth_endpoint = "https://eth-mainnet.gateway.pokt.network/v1/lb/00000000000000000000000"
    headers = {'content-type': 'application/json'}

    payload = {
        "method": "eth_getTransactionCount",
        "params": [address, "latest"],
        "jsonrpc": "2.0",
        "id": 1,
    }
    response = requests.post(pokt_eth_endpoint, data=json.dumps(payload), headers=headers).json()
    transaction_count = int(response['result'], 16)
    return transaction_count


def get_balance(address):
    pokt_Eth_endpoint = "https://eth-mainnet.gateway.pokt.network/v1/lb/00000000000000000000000"
    headers = {'content-type': 'application/json'}

    payload = {
        "method": "eth_getBalance",
        "params": [address, "latest"],
        "jsonrpc": "2.0",
        "id": 1,
    }
    response = requests.post(pokt_Eth_endpoint, data=json.dumps(payload), headers=headers).json()
    balance = int(response['result'], 16)
    balance_in_eth = balance/10**18
    return balance_in_eth


In [ ]:
data = pd.DataFrame([[],[],[],[],[],[],[],[]]).transpose()
data.rename(columns={0: 'source_wallet', 1: 'txn_count',2:'No_Days_Active',3:'initial_funding',4:'Last_txn_out',5:'balance',6:'first_active_date',7:'last_trasaction_out'},inplace=True)

In [ ]:
def Query_data(dataframe):
    for address in votes.source_wallet.unique():
        pseudo_dataframe = []
        no_grants_funded = len(dataframe['No_Grants_Funded'].loc[dataframe.source_wallet == address])
        if no_grants_funded <= 5:
            print(1,address)
            noda,inital_fund,lto,fad,lad = get_transactions_and_date_info(address)
            txn_count = get_txn_count(address)
            balance =get_balance(address)
            pseudo_dataframe.append(address)
            pseudo_dataframe.append(txn_count)
            pseudo_dataframe.append(noda)
            pseudo_dataframe.append(inital_fund)
            pseudo_dataframe.append(lto)
            pseudo_dataframe.append(balance)
            pseudo_dataframe.append(pd.to_datetime(fad))
            pseudo_dataframe.append(pd.to_datetime(lad))
            print(1,address)
            data.loc[len(data.index)] = pseudo_dataframe
            time.sleep(1.5)
        else:
            print(2)


In [ ]:
Query_data(votes)

In [ ]:
data.to_csv('data.csv',index=False)